In [35]:
import numpy as np
import cv2
import os
from os import listdir
from os.path import isfile, join
import json
import sys
import numpy
from PIL import Image
from math import pi,sin,cos,tan,atan2,hypot,floor,sqrt
from numpy import clip,pi
from __future__ import print_function
# import os.pathback
import zipfile
import timeit
import psycopg2
import hashlib
import argparse


In [ ]:
parser = argparse.ArgumentParser(description='frame extraction & cubic projection')
parser.add_argument('-f', '--framelist', default='lazare_frame_list',
                    help='frame list table name')
parser.add_argument('-s', '--schema', default='gare_st_lazare', 
                    help='schema for the station')
parser.add_argument('-i', '--imagelist', default='images_225', 
                    help='image list table name')
parser.add_argument('-v', '--video', default='../data/videos/lazare.mp4', 
                    help='video path')
parser.add_argument('-o', '--output', default='../data/images/lazare_225', 
                    help='output image path')
parser.add_argument('-d', '--direction', default=22.5, type=float, 
                    help='direction as degree')

args = parser.parse_args()

In [44]:
# initial variables
deviation = [20.9, 16.8, 160.2]

# variable from input
frame_list_tablename = args.framelist
schema = args.schema
img_list_dbname = args.imageslist
video = args.path
root_dir= args.output
orientation = args.direction

In [45]:
def create_table (schema, img_list_dbname, conn, cur):
    query = '''
    DROP TABLE IF EXISTS %s.%s;
    CREATE TABLE %s.%s
    (
      img_id text,
      time_abs text,
      path text,
      orientation text,
      category text
    )
    ''' % (schema, img_list_dbname, schema, img_list_dbname)
    cur.execute(query)
    conn.commit()

In [50]:
def capture(root_dir, zone_id, timestamp, count, cap):
   
    # capture frame at certain millsection from frame_no
    # video_name is the video being called
    try:
        cap.set(cv2.CAP_PROP_POS_MSEC, timestamp + 190420); # Where frame_no is the frame you want
        ret, frame = cap.read() # Read the frame
        
        output_path = os.path.join(root_dir, zone_id)
        hash_id = hashlib.sha224(os.path.join(output_path, 'pano', "%s.png" % count)).hexdigest()
        fn = os.path.join(output_path, 'pano', "%s.png" % hash_id)
        
        cv2.imwrite(fn, frame)
    
    except:
        print("Unexpected error:", sys.exc_info()[0])
        raise

    return fn

In [47]:
def generate_mapping_data(image_width, rotation):
    
    # transform pano to cubic picture
    in_size =[image_width, int(image_width * 3 / 4)]
    edge = in_size[0] // 4  # The length of each edge in pixels

    # Create numpy arrays
    out_pix = numpy.zeros((in_size[1], in_size[0], 2), dtype="f4")
    xyz = numpy.zeros((in_size[1] * in_size[0] // 2, 3), dtype="f4")
    vals = numpy.zeros((in_size[1] * in_size[0] // 2, 3), dtype="i4")
    
    start, end = 0, 0
    rng_1 = numpy.arange(0, edge * 3)
    rng_2 = numpy.arange(edge, edge * 2)
    for i in range(in_size[0]):
        # 0: back
        # 1: left
        # 2: front
        # 3: right
        face = int(i / edge)
        rng = rng_1 if face == 2 else rng_2
 
        end += len(rng)
        vals[start:end, 0] = rng
        #print("vals", vals[start:end, 0])
        vals[start:end, 1] = i        
        vals[start:end, 2] = face
        start = end
    
    # Top/bottom are special conditions
    j, i, face = vals.T
    face[j < edge] = 4  # top
    face[j >= 2 * edge] = 5  # bottom
 
    # Convert to image xyz
    a = 2.0 * i / edge
    b = 2.0 * j / edge
    one_arr = numpy.ones(len(a))
    for k in range(6):
        face_idx = face == k
 
        # Using the face_idx version of each is 50% quicker
        one_arr_idx = one_arr[face_idx]
        a_idx = a[face_idx]
        b_idx = b[face_idx]
 
        if k == 0:
           vals_to_use =  [-one_arr_idx, 1.0 - a_idx, 3.0 - b_idx]
        elif k == 1:
           vals_to_use =  [a_idx - 3.0, -one_arr_idx, 3.0 - b_idx]
        elif k == 2:
           vals_to_use =  [one_arr_idx, a_idx - 5.0, 3.0 - b_idx]
        elif k == 3:
           vals_to_use =  [7.0 - a_idx, one_arr_idx, 3.0 - b_idx]
        elif k == 4:
           vals_to_use =  [b_idx - 1.0, a_idx - 5.0, one_arr_idx]
        elif k == 5:
           vals_to_use =  [5.0 - b_idx, a_idx - 5.0, -one_arr_idx]
 
        xyz[face_idx] = numpy.array(vals_to_use).T
 
    # Convert to theta and pi
    x, y, z = xyz.T
    theta = numpy.arctan2(y, x)+rotation*pi/180
    r = numpy.sqrt(x**2 + y**2)
    phi = numpy.arctan2(z, r)
 
    # Source img coords
    uf = (2.0 * edge * (theta + pi) / pi) % in_size[0]
    uf[uf==in_size[0]] = 0.0 # Wrap to pixel 0 
    vf = (2.0 * edge * (pi / 2 - phi) / pi)
 
    # Mapping matrix
    out_pix[j, i, 0] = vf
    out_pix[j, i, 1] = uf
 
    map_x_32 = out_pix[:, :, 1]
    map_y_32 = out_pix[:, :, 0]
    return map_x_32, map_y_32
    

In [48]:
def seperate(root_dir, imgOut, pic_dir, count, time_abs, zone_id, rotate_45):
    
    output_path = os.path.join(root_dir, zone_id)
    
    # cut cubic map to 6 directions. imgOut is picture from pano_to_cub
    if rotate_45 == False:
        name_map = [ 
             ["", "", "top", ""],
             ["back", "left", "front", "right"],
             ["", "", "bottom", ""]]
    else:
        name_map = [ 
             ["", "", "top", ""],
             ["back_right", "front_right", "front_left", "back_left"],
             ["", "", "bottom", ""]]
    width, height = imgOut.size
    cube_size = width / 4
    
    # save images from different directions
    for row in range(3):
        for col in range(4):
            if name_map[row][col] != "":
                sx = cube_size * col
                sy = cube_size * row
                hash_id = hashlib.sha224(os.path.join(output_path, name_map[row][col], "%s.png" % count)).hexdigest()
                fn = os.path.join(output_path, name_map[row][col], "%s.png" % hash_id)
#                 print("%s --> %s" % (str((sx, sy, sx + cube_size, sy + cube_size)), fn))
                if not (name_map[row][col] == 'top' or name_map[row][col] == 'bottom'):
                    imgOut.crop((sx, sy, sx + cube_size, sy + cube_size)).save(fn) 
                    "save to dataset"
                    query = '''insert into %s.%s values('%s', '%s', '%s', '%s', '%s')''' % (schema, img_list_dbname, hash_id, time_abs, fn, name_map[row][col], zone_id)
                    cur.execute(query)
                    conn.commit()

In [ ]:
if __name__ == '__main__':
    
    # connect to db
    conn_string = "host='localhost' dbname='indoor_position' user='postgres' password='tiancai' port='5432'"
    conn = psycopg2.connect(conn_string)
    cur = conn.cursor()
    
    # create table to store information about exported images
    create_table(schema, img_list_dbname, conn, cur)

    # retrieve frame list
    query = 'select * from %s.%s order by time_relative asc' % (schema, frame_list_tablename)
    cur.execute(query)
    conn.commit()
    results = cur.fetchall()

    # start = timeit.default_timer()
    count = 0
    cap = cv2.VideoCapture(video)
    
    # iterate through images in frame list
    for idx, image in enumerate(results):
        zone_id   = image[0].split('_')[1] + '_' + image[3]
        timestamp = int(str(image[2]).split('.')[0])
        time_abs  = image[1]
        
        output_path = os.path.join(root_dir, zone_id)
        if not os.path.exists(output_path):
            os.makedirs(os.path.join(output_path, 'pano'))
            os.makedirs(os.path.join(output_path, 'left'))
            os.makedirs(os.path.join(output_path, 'right'))
            os.makedirs(os.path.join(output_path, 'front'))
            os.makedirs(os.path.join(output_path, 'back'))
            os.makedirs(os.path.join(output_path, 'front_left'))
            os.makedirs(os.path.join(output_path, 'back_left'))
            os.makedirs(os.path.join(output_path, 'back_right'))
            os.makedirs(os.path.join(output_path, 'front_right'))

        # extract&save frame
        frame_captured = capture(root_dir, zone_id, timestamp, count, cap)

        # projection transform
        imgIn  = Image.open(frame_captured)
                
        inSize = imgIn.size
        map_x_32, map_y_32 = generate_mapping_data(inSize[0], image[4] + orientation)

        cubemap = cv2.remap(numpy.array(imgIn), map_x_32, map_y_32, cv2.INTER_LINEAR) 
        imgOut = Image.fromarray(cubemap) # project the transformed pixel back to cubemap
        
        seperate(root_dir, imgOut, frame_captured, count, time_abs, zone_id, False) #3.cut
        count += 1
        
        # map_x_32, map_y_32 = generate_mapping_data(inSize[0], image[4] + 4.4 + 45)
        map_x_32, map_y_32 = generate_mapping_data(inSize[0], image[4] + 4.4 + orientation + 22.5)
        cubemap = cv2.remap(numpy.array(imgIn), map_x_32, map_y_32, cv2.INTER_LINEAR) 
        imgOut = Image.fromarray(cubemap) # project the transformed pixel back to cubemap
        seperate(root_dir, imgOut, frame_captured, count, time_abs, zone_id, True) #3.cut
        count += 1
        
        sys.stdout.write('\rzone id: %s, count: %s' % (zone_id, count))
        sys.stdout.flush()
        
        # stop = timeit.default_timer()
        # print(stop - start)


zone id: second_9, count: 170464